![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [138]:
import pandas as pd
import numpy as np
import datetime
from time import strptime
import calendar

# Start coding here...

#Import the bank_marketing.csv as a DataFrame

banking = pd.read_csv('bank_marketing.csv')
banking['year'] = 2022
banking.month = banking.month.str.title()
 
month_dict = dict((month, index) for index, month in enumerate(calendar.month_abbr) if month)
banking['month'] = banking['month'].map(month_dict)


In [139]:
#Create the three DataFrames: client, campaign, and economics

client = banking[['client_id','age','job','marital','education','credit_default','mortgage']]
campaign = banking[['client_id','number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome','campaign_outcome']] #don't forget to add the datetime column later
economics = banking[['client_id','cons_price_idx', 'euribor_three_months']]

In [140]:
#Clean client DataFrame

#change "." to "_" in job and education columns

client['job']=client['job'].str.replace('.', '_')
client['education']=client['education'].str.replace('.', '_')

#replace unknown in the education column with np.NaN

client['education'].replace(to_replace={'unknown': np.nan}, inplace=True)

client['credit_default'].replace(to_replace={'unknown': np.nan}, inplace=True)
client['credit_default']=client['credit_default'].replace({'yes':True, 'no':False})
client['mortgage']=client['mortgage'].replace({'yes':True, 'no':False})

client['credit_default']=client['credit_default'].astype(bool)
client['mortgage']=client['mortgage'].astype(bool)





In [141]:
#Clean campaign DataFrame

#convert previous_outcome and campaign_outcome to boolean dtypes
campaign['previous_outcome']=campaign['previous_outcome'].replace({'nonexistent':True, 'failure':False})
campaign['campaign_outcome']=campaign['campaign_outcome'].replace({'yes':True, 'no':False})

campaign['previous_outcome']=campaign['previous_outcome'].astype(bool)
campaign['campaign_outcome']=campaign['campaign_outcome'].astype(bool)

#create new column called 'last_contact_date' and create a datetime from day, month, year columns
campaign['last_contact_date']=pd.to_datetime(banking[['year','month', 'day']])
# campaign['last_contact_date']=pd.to_datetime(campaign['last_contact_date']).dt.strftime('%Y-%m-%d')

In [142]:
#Save three DataFrames as csv files

client.to_csv('client.csv')
campaign.to_csv('campaign.csv')
economics.to_csv('economics.csv')